In [2]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
from urllib.request import Request, urlopen
import time
from selenium.common.exceptions import NoSuchElementException

# gadgets.ndtv.com
def getReviews(list_of_page):
    reviews = []
    for page_url in list_of_page:
        page = requests.get(page_url)
        if page.status_code==200:   # Check if success
            soup = BeautifulSoup(page.content, 'html.parser')
            raw_header = soup.find('h1')
            header = raw_header.get_text().strip()
            raw_review = soup.find('div', {'class': 'content_text row description'}) 
            review = raw_review.get_text().strip()
            
            if 'Review' not in header:
                if 'First Impressions' in header:
                    name = header.split(" First Impressions")[0]
                else:
                    name = header.split(" review")[0]
            else:
                name = header.split(" Review")[0]

            reviews.append({ 'Name': name, 'Headline': header, 'Review': review, 
                            'Source': 'gadgets.ndtv', 'URL': page_url})
    df = pd.DataFrame(reviews)
    return df

def getListOfPhone(page_url):
    links = []
    page = requests.get(page_url) # request current page
    soup = BeautifulSoup(page.content, 'html.parser') # parse HTML content
    links = fetchLinks(soup, links)
    count = 1
    while count <= 50: # Site max always have 50 page 
        time.sleep(0.05)
        next_link = page_url + str(count)
        page = requests.get(next_link) # request current page
        soup = BeautifulSoup(page.content, 'html.parser') # parse HTML content
        links = fetchLinks(soup, links)
        count = count + 1
    return links  

def fetchLinks(soup, links):
    divs = soup.find_all('div', {'class': 'caption_box'})
    for div in divs:
        anchors = div.find('a')
        if 'https://' in anchors.get('href'):
            links.append(anchors.get('href'))
            
    return links

page_url = 'https://gadgets.ndtv.com/mobiles/reviews/page-'
list_of_page = getListOfPhone(page_url)

df = getReviews(list_of_page)
df.to_csv('CSVs/gadgets-ndtv.csv')
df

,Name,Headline,Review,Source,URL
0,Realme 8i,Realme 8i Review: All-Rounder on a Budget?,Realme 8i was launched in India as the success...,gadgets.ndtv,https://gadgets.ndtv.com/mobiles/reviews/realm...
1,Samsung Galaxy A22 5G,Samsung Galaxy A22 5G Review: Very Forgettable,The Samsung Galaxy A22 5G launched in India ar...,gadgets.ndtv,https://gadgets.ndtv.com/mobiles/reviews/samsu...
2,Infinix Hot 11S,Infinix Hot 11S Review: A Decent Gaming-Focuss...,There isn't much choice if you are looking to ...,gadgets.ndtv,https://gadgets.ndtv.com/mobiles/reviews/infin...
3,Realme Narzo 50A,"Realme Narzo 50A Review: Big on Battery Life, ...","Launched in February this year, Realme's Narzo...",gadgets.ndtv,https://gadgets.ndtv.com/mobiles/reviews/realm...
4,Android 12,"Android 12 First Impressions: Fresh, New and P...","Compared to iOS 14 or 15, Android 12 is mainly...",gadgets.ndtv,https://gadgets.ndtv.com/mobiles/reviews/andro...
...,...,...,...,...,...
1015,Yu Yuphoria:,Yu Yuphoria: First Impressions,Micromax subsidiary Yu Televentures released i...,gadgets.ndtv,https://gadgets.ndtv.com/mobiles/reviews/yu-yu...
1016,BlackBerry Leap,"BlackBerry Leap Review: For Some, a Leap of Faith",It seemed for a long time as though this day m...,gadgets.ndtv,https://gadgets.ndtv.com/mobiles/reviews/black...
1017,Honor 4C:,Honor 4C: First Impressions,"At a glitzy launch event in Mumbai on Friday, ...",gadgets.ndtv,https://gadgets.ndtv.com/mobiles/reviews/honor...
1018,Lenovo A7000,Lenovo A7000 Review: Multimedia Phablet on a B...,"As phones started becoming smart, people disco...",gadgets.ndtv,https://gadgets.ndtv.com/mobiles/reviews/lenov...
